In [ ]:
import csv
import os
import json
import time
import random
import pyodbc    # pip install pyodbc

# 1. Connect to Azure SQL Database

In [ ]:
# Define database connection details
server   = 'AZURE_SQL_DATABASE_SERVER_NAME'
database = 'AZURE_SQL_DATABASE_DATABASE_NAME'
username = 'AZURE_SQL_DATABASE_USERNAME'
password = 'AZURE_SQL_DATABASE_PASSWORD'
driver   = '{ODBC Driver 17 for SQL Server}'

# Connect to Azure SQL Database
connection = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password};'
)

# Create a cursor
cursor = connection.cursor()

# 2. Delete all data sets in Azure SQL Database by stored procedure

In [ ]:
# Delete all datasets in database by using stored procedure
cursor.execute("EXEC SP_DeleteAllInspectionData")
connection.commit()

# 3. Insert 10.000 new data sets in Azure SQL Database

In [ ]:
# The variables are required to read the corresponding lines from the CSV sequence file.
# For this use case, the first 10,000 lines are read.
sequence_range_start = 0
sequence_range_end   = 10000

# Define lists to store the values from the sequence files that fall within the defined range.
order_numbers, serial_numbers, article_names, machine_names = [], [], [], []

# Read values from the CSV sequence file
with open('sequence_of_inserting_data.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row

    # Skip lines until the start of the defined range
    for _ in range(sequence_range_start):
        next(reader)

    # Read lines within the defined range
    for _ in range(sequence_range_end - sequence_range_start):
        row = next(reader)
        order_numbers.append(row[0])
        serial_numbers.append(row[1])
        article_names.append(row[2])
        machine_names.append(row[3])

# Zip the individual lists together for easier handling
sequence_of_inserting_data = zip(order_numbers, serial_numbers, article_names, machine_names)

# Insert 10,000 new datasets into the Azure SQL Database
for order_number, serial_number, article_name, machine_name in sequence_of_inserting_data:
    
    # Prepare the dataset 
    
    # Read the reference dataset for the article
    reference_dataset_file_path = os.path.join('Reference_Datasets', 
                                               f"reference_dataset_{article_name}.json")
    with open(reference_dataset_file_path, 'r') as file:
        json_data = json.load(file)

    # Replace data in JSON string for Azure Cosmos DB
    json_data.update({
        'OrderNumber': order_number,     # Use value from the sequence file
        'SerialNumber': serial_number,   # Use value from the sequence file
        'MachineName': machine_name,     # Use value from the sequence file
    })

    # Generate the MeasuredValue for each inspection step
    for inspection in json_data['InspectionsAndResults']:
        lower_border_value = float(inspection['InspectionLowerBorderValue'])
        upper_border_value = float(inspection['InspectionUpperBorderValue'])

        measured_value = str(round(random.uniform(lower_border_value, upper_border_value), 2))
        inspection['InspectionResultMeasuredValue'] = measured_value

    # Execute the insert operation 

    # Convert data to JSON string
    json_data_as_string = json.dumps(json_data)

    # Insert the dataset into the Azure SQL Database by executing the stored procedure
    cursor.execute("{CALL SP_InsertInspectionOperation('" + json_data_as_string + "')}")


# 4. Retrieve all currently available serialnumbers in Azure SQL Database

In [ ]:
# Retrieve all serialnumbers from Azure SQL Database and store it in a list
cursor.execute('SELECT DISTINCT SerialNumber FROM InspectionOperations')
serialnumbers = [row.SerialNumber for row in cursor.fetchall()]

# 5. Run the delete operation

In [ ]:
# The database contains 10.000 data sets
number_of_all_datasets = 10000

# Declare list to store mean query durations per iteration
query_durations = []

In [ ]:
# Declare 10 iterations a 1.000 data sets
for _ in range(10):

    # Declare a list to store the mean operation durations of an iteration (1.000 Datasets)
    iteration_durations = []
    
    # 1.000 data sets are deleted in each delete iteration
    for _ in range(1000):
        
        # Select a random serial number for which the corresponding data record is to be deleted
        random_serialnumber = random.choice(serialnumbers)

        # Run the CRUD-Operation
    
        # Record the current timestamp before running the operation  
        query_start_time = time.time()

        # Execute the delete operation and commit
        cursor.execute(f'DELETE FROM InspectionOperations WHERE SerialNumber = "{random_serialnumber}"')
        connection.commit()
        
        # Record the current timestamp after running the operation
        query_end_time = time.time()

        # Calculate the duration time for this operation and append this to list
        query_duration = query_end_time - query_start_time
        iteration_durations.append(query_duration)

        # Remove the affected serial number from the list    
        serialnumbers.remove(random_serialnumber)

    # Calculate the mean duration for the iteration of 1.000 data sets and store in query_durations list        
    mean_duration = sum(iteration_durations) / len(iteration_durations)
    query_durations.append([mean_duration, 1, number_of_all_datasets])
    
    number_of_all_datasets -= 1000

# 6. Saving the recorded operation times in the CSV result file

In [ ]:
# Define the file path for the CSV file in the "Experiment_Results" directory
filepath = os.path.join("Experiment_Results", "delete_data.csv")

# Check if the file already exists
file_exists = os.path.isfile(filepath)

# Open the CSV file in append mode; create the file if it does not exist
with open(filepath, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # If the file does not exist, write the header row
    if not file_exists:
        writer.writerow(['DurationTime', 'NumberOfProcessedDatasets', 'NumberOfDatasetsInDatabase'])
    
    # Write the rows of data from the query_durations list
    writer.writerows(query_durations)